In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")

In [3]:
df.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
208,ham,You please give us connection today itself bef...,NaN,NaN,NaN
331,ham,Maybe i could get book out tomo then return it...,NaN,NaN,NaN
401,ham,Dear how is chechi. Did you talk to her,NaN,NaN,NaN
5393,ham,"All done, all handed in. Don't know if mega sh...",NaN,NaN,NaN
4089,spam,We tried to call you re your reply to our sms ...,NaN,NaN,NaN


In [4]:
df.shape

(5572, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [7]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)

,target,text
1239,ham,Hope you are having a great new semester. Do w...
3091,ham,"Dear, take care. I am just reaching home.love ..."
3404,ham,"Jus chillaxin, what up"
2109,ham,Lara said she can loan me &lt;#&gt; .
4702,ham,Mmmmmmm *snuggles into you* ...*deep contented...


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])

In [9]:
df = df.drop_duplicates(keep='first')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5169 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  5169 non-null   int32 
 1   text    5169 non-null   object
dtypes: int32(1), object(1)
memory usage: 101.0+ KB


In [11]:
import nltk
print(nltk.__version__)
nltk.download('punkt')
nltk.download('stopwords')

3.8.1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [13]:
def transform_text(text): 
    text = text.lower() # chuyển thành chữ thường
    text = nltk.word_tokenize(text) # chia text thành mảng các từ riêng lẻ
    
    y = []
    for i in text:
        if i.isalnum(): # kiểm tra từ chỉ bao gồm chữ và số
            y.append(i)
    
    text = y[:] # copy y to text
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation: # kiểm tra xem từ không phải là từ dừng và dấu câu
            y.append(i)
            
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i)) # cắt giảm từ trở về dạng gốc
            
    return " ".join(y)

In [14]:
transform_text('Whoever has money, please donate to me')

'whoever money pleas donat'

In [15]:
df['transformed_text'] = df['text'].apply(transform_text)

In [16]:
df.head()

,target,text,transformed_text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
cv = CountVectorizer()   #Vector hóa văn bản
tfidf = TfidfVectorizer(max_features = 3000) # chuyển đổi văn bản thành vectơ TF-IDF giữ lại 3000 đặc trưng

In [18]:
X = tfidf.fit_transform(df['transformed_text']).toarray() # chuyển đổi cột 'transformed_text' thành ma trận các vectơ TF-IDF
X.shape

(5169, 3000)

In [19]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X) # chuẩn hóa ma trận về khoảng giá trị [0, 1]
X.shape

(5169, 3000)

In [20]:
y = df['target'].values

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, classification_report, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots

# Repeated Hold-out

In [28]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
param_grid_knn = {'n_neighbors': range(1,20,1)}
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=rskf, scoring='accuracy')
grid_search_knn.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=42),
             estimator=KNeighborsClassifier(n_neighbors=2),
             param_grid={'n_neighbors': range(1, 20)}, scoring='accuracy')

In [30]:
best_params = grid_search_knn.best_params_  # Best hyperparameters
best_score = grid_search_knn.best_score_   # Best cross-validation score
best_estimator = grid_search_knn.best_estimator_  # Model with best parameters

In [31]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=20)
param_grid_dtc = {'max_depth': range(1,30,1)}
grid_search_dtc = GridSearchCV(dtc, param_grid_dtc, cv=rskf, scoring='accuracy')
grid_search_dtc.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=42),
             estimator=DecisionTreeClassifier(max_depth=20),
             param_grid={'max_depth': range(1, 30)}, scoring='accuracy')

In [32]:
best_params = grid_search_dtc.best_params_  # Best hyperparameters
best_score = grid_search_dtc.best_score_   # Best cross-validation score
best_estimator = grid_search_dtc.best_estimator_  # Model with best parameters

In [33]:
print(best_params)
print(best_score)
print(best_estimator)

{'max_depth': 27}
0.956148954270517
DecisionTreeClassifier(max_depth=27)


In [34]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=40)
param_grid_rfc = {'n_estimators': range(10,120,10)}
grid_search_rfc = GridSearchCV(rfc, param_grid_rfc, cv=rskf, scoring='accuracy')
grid_search_rfc.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=42),
             estimator=RandomForestClassifier(n_estimators=40),
             param_grid={'n_estimators': range(10, 120, 10)},
             scoring='accuracy')

In [35]:
best_params = grid_search_rfc.best_params_  # Best hyperparameters
best_score = grid_search_rfc.best_score_   # Best cross-validation score
best_estimator = grid_search_rfc.best_estimator_  # Model with best parameters

In [36]:
print(best_params)
print(best_score)
print(best_estimator)

{'n_estimators': 100}
0.9752369111393642
RandomForestClassifier()


In [38]:
from sklearn.svm import SVC
svc = SVC(kernel='sigmoid')
param_grid_svc = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']}
grid_search_svc = GridSearchCV(svc, param_grid_svc, cv=rskf, scoring='accuracy')
grid_search_svc.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=42),
             estimator=SVC(kernel='sigmoid'),
             param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [39]:
best_params = grid_search_svc.best_params_  # Best hyperparameters
best_score = grid_search_svc.best_score_   # Best cross-validation score
best_estimator = grid_search_svc.best_estimator_  # Model with best parameters

In [40]:
print(best_params)
print(best_score)
print(best_estimator)

{'kernel': 'linear'}
0.9796222404057463
SVC(kernel='linear')


In [41]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [42]:
gnb = GaussianNB()
scores = cross_val_score(gnb, X, y, cv=rskf, scoring='accuracy')

In [43]:
print(scores)
print("Mean accuracy:", scores.mean())
print("Standard deviation:", scores.std())

[0.86653772 0.86460348 0.85976789 0.84816248 0.85769603 0.86073501
 0.86653772 0.86653772 0.87137331 0.85866409 0.86750484 0.84526112
 0.87814313 0.86266925 0.86447241]
Mean accuracy: 0.8625777454884992
Standard deviation: 0.00798652859168333


In [44]:
mnb = MultinomialNB()
scores = cross_val_score(mnb, X, y, cv=rskf, scoring='accuracy')
print("Standard deviation:", scores.std())

Standard deviation: 0.002629889757848197


In [45]:
print(scores)
print("Mean accuracy:", scores.mean())

[0.98065764 0.98355899 0.98065764 0.98259188 0.98257502 0.98452611
 0.9787234  0.98355899 0.98452611 0.97870281 0.98452611 0.97678917
 0.98065764 0.97969052 0.98644724]
Mean accuracy: 0.9818792859492332


In [46]:
bnb = BernoulliNB()
scores = cross_val_score(bnb, X, y, cv=rskf, scoring='accuracy')

In [47]:
print(scores)
print("Mean accuracy:", scores.mean())
print("Standard deviation:", scores.std())

[0.98162476 0.98452611 0.97485493 0.9787234  0.98257502 0.97969052
 0.97969052 0.97969052 0.98259188 0.98354308 0.98162476 0.97388781
 0.98452611 0.97678917 0.98063892]
Mean accuracy: 0.9803318347529592
Standard deviation: 0.003130138856736625


In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
lg_model = LogisticRegression(max_iter=200)
scores = cross_val_score(lg_model, X, y, cv=rskf, scoring='accuracy')

In [50]:
print(scores)
print("Mean accuracy:", scores.mean())
print("Standard deviation:", scores.std())

[0.97098646 0.97485493 0.96905222 0.97388781 0.9767667  0.96421663
 0.97678917 0.97582205 0.9729207  0.97289448 0.97388781 0.97195358
 0.97195358 0.96615087 0.97483059]
Mean accuracy: 0.9724645062392996
Standard deviation: 0.0035259185696393724
